In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [123]:
test2 = pd.read_csv('./SE_rents2018_test2.csv', index_col=0)
test1 = pd.read_csv('./SE_rents2018_test1.csv', index_col=0)
train1 = pd.read_csv('./SE_rents2018_train.csv', index_col=0)

In [124]:
train1.sample(5)

,addr_unit,building_id,bedrooms,bathrooms,size_sqft,created_at,addr_street,addr_city,addr_zip,addr_lat,...,has_garden,has_childrens_playroom,rent,no_fee,description,neighborhood,borough,unit,floornumber,line
rental_id,,,,,,,,,,,,,,,,,,,,,
7298014,#1R,391426,4,2.0,1350.0,2018-06-22 11:26:28,208 STARR STREET,Brooklyn,11237,40.704810,...,0,0,5499,1,Location Location Location!\r\nPrime Bushwick\...,Bushwick,Brooklyn,1R,1.0,R
7398274,#5Q,361810,0,1.0,403.0,2018-07-24 16:05:53,26 WEST STREET,Brooklyn,11222,40.727250,...,0,0,2625,1,5th floor Studio with private balcony becoming...,Greenpoint,Brooklyn,5Q,5.0,Q
7410316,#3,201967,2,1.0,1000.0,2018-07-25 21:42:46,83 ST MARK'S PLACE,Brooklyn,11217,40.681794,...,1,0,2999,0,PARK SLOPE NORTH\r\nMinutes Atlantic Terminal ...,Park Slope,Brooklyn,3,3.0,NaN
7228219,#3LL,24151,1,1.0,785.0,2018-06-01 13:28:34,85 FOURTH AVENUE,New York,10003,40.731503,...,1,0,4450,1,*No Fee and Deposit Free with Rhino Protection...,East Village,Manhattan,3LL,3.0,LL
7294957,#2C,1265863,2,1.0,650.0,2018-06-21 14:06:34,28-46 34 STREET,Astoria,11103,40.766284,...,0,0,2295,1,**HEART OF ASTORIA** **RENOVATED** This true 2...,Astoria,Queens,2C,2.0,C


In [125]:
feature_cols = [
    'bedrooms', 'bathrooms', 'floornumber', 
    'has_childrens_playroom', 'has_pool', 
    'has_concierge', 'has_garage', 'has_gym', 
    'is_furnished','allows_pets'
]
train_features = train1[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train1['rent']

In [126]:
# using linear regression method
lreg = LinearRegression()
lreg.fit(train_features, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [127]:
test_features = test1[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test1['predicted'] = lreg.predict(test_features)
print("Mean Squared Error for Test1(Linear Regression):",mean_squared_error(test1['rent'], test1['predicted']))

Mean Squared Error for Test1(Linear Regression): 4529358.208464365


In [128]:
rf = RandomForestRegressor()
rf.fit(train_features, train_target)

# construct predictions 
test1['predicted'] = rf.predict(test_features)
print("Mean Squared Error for Test1(Random Forest Regressor):",mean_squared_error(test1['rent'], test1['predicted']))

Mean Squared Error for Test1(Random Forest Regressor): 3690651.8474690663


/Users/Oscar/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [129]:
#Combine data to increase total training data
master_df = train1.append(test1, sort=False)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

rf.fit(master_features, master_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [130]:
#handling missing data
test2_features = test2[feature_cols].fillna(master_df[feature_cols].median(), axis=0)

test2['rent'] = rf.predict(submit1_features)
test2['rent'].to_csv('test2.csv', header=True)

test2['median_rent'] = np.ones(test2['rent'].shape) * master_target.median()
print("Mean Squared Error for Test2: ",mean_squared_error(test2['rent'], test2['median_rent']))

Mean Squared Error for Test2:  6588950.597649638
